<a href="https://colab.research.google.com/github/suvamgurung/MTech-CS-Project-2022/blob/main/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using "43_sent_data_sept_22" csv
- Tokenize the sentence
- Remove Stop Words
- Perform Lemmatization
- Extract Entities and Relations


#Uploading the csv data-file containing the refined Sentences related to PM-JAY as Pandas Dataframe from the Google Drive


In [ ]:
import pandas as pd
#data= pd.read_csv("/content/drive/MyDrive/Project/43_sent_data_sept_22.csv")
data= pd.read_csv("/content/drive/MyDrive/Project/refined_sentences.csv")

#Importing the spacy library
The **displacy** helps to render the dependencies of each sentences\
loading "**en_core_web_sm**" will give us the NLP model to carry-on NLP on sentences.

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
from spacy import displacy
nlp= spacy.load("en_core_web_sm")
#nlp= spacy.load("en_core_web_lg")
#for sent in data['sentences']:
#    print(sent)

#Printing the Text, POS, Tag, Depencency and Head for each token (word) in ever sentence. Can be used to find the logic for extraction of entities and relations.

In [ ]:
from tabulate import tabulate
i=1
for sent in data['sentences']:
    table_data=[]
    d= nlp(sent)
    for tok in d:
        table_data.append([tok.text, tok.pos_,tok.tag_,tok.dep_,tok.head])
    print('\033[1m',i,')'+sent+'\033[0m')        
    print(tabulate(table_data, headers=['Token_text','POS','Tag','Dependency','Head']))
    print('\n')
    i+=1

 1 )The Patient approaches the EHCP Registration Desk
Token_text    POS    Tag    Dependency    Head
------------  -----  -----  ------------  ----------
The           DET    DT     det           Patient
Patient       PROPN  NNP    nsubj         approaches
approaches    VERB   VBZ    ROOT          approaches
the           DET    DT     det           Desk
EHCP          PROPN  NNP    compound      Desk
Registration  PROPN  NNP    compound      Desk
Desk          PROPN  NNP    dobj          approaches


 2 )The Patient approaches the PM-JAY Kiosk
Token_text    POS    Tag    Dependency    Head
------------  -----  -----  ------------  ----------
The           DET    DT     det           Patient
Patient       PROPN  NNP    nsubj         approaches
approaches    VERB   VBZ    ROOT          approaches
the           DET    DT     det           Kiosk
PM            PROPN  NNP    compound      Kiosk
-             PUNCT  HYPH   punct         Kiosk
JAY           PROPN  NNP    compound      Kiosk
Ki

In [ ]:
t1="The EHCP provides the relevant medication and diagnostics as per the package upto 15 days to the PM-JAY Beneficiary"
t2="The EHCP provide the relevant medication and diagnostics as per the package upto 15 days to the PM-JAY Beneficiary"
f= nlp(t1)
g= nlp(t2)
table_data2=[]
for tok in f:
    table_data2.append([tok.text, tok.pos_,tok.tag_,tok.dep_,tok.head])
print('\033[1m'+sent+'\033[0m')        
print(tabulate(table_data2, headers=['Token_text','POS','Tag','Dependency','Head']))
print('\n')
table_data2=[]
for tok in g:
    table_data2.append([tok.text, tok.pos_,tok.tag_,tok.dep_,tok.head])
print('\033[1m'+sent+'\033[0m')        
print(tabulate(table_data2, headers=['Token_text','POS','Tag','Dependency','Head']))
print('\n')

## Now since we have the dependencies of each and every word, we need to extract entities and relations based on these depedencies. 
## What are to be extracted are as follows :-
**1.   Source Entity:-** There can be one or more source entity/ies depending upon the sentence. Usually the source entity will be the found as the first nsubj dependencies. If there is a complex sentence, the next source source entity usually is the first Target Entity and so on.\
**2.   Target Entity:-** There can be one or more Target Entity/ies depending upon the complexity of the sentence. The Target Entity is usually determined by dobj or pobj.\
**3.   Relation:-** There can be only one relation between a Source and a Target Entity, but in a sentence there can be one or more relations depending upon the number of source and target pairs. The relations are usually the ROOT and prep.\
\
**We do need to concatinate the dets, modifiers etc with their respective nsubj, dobj or pobj**
\
##Firstly, find the logic to Extract the Source Entity
##Secondly, find the logic to extract Relation
##Thirdly, find the logic to extract Target Entity 
**Do keep in mind that the entities and relation must be extracted as a triple for each sentence !!!**


In [ ]:
#function to extract Triple (Source, Relation and Target) from each sentence
#firstly find the ROOT along with its prefixes and suffixes
#secondly find the Source Entity and the Target Phrase which may contain in itself one or more set of SE,R,TE
def getTriple(sent):
    se="" #Source Entity
    r="" #Relation
    te="" #Target Entity
########################################################
    root_prefix=""
    root_suffix=""
    root_word=""
    combined_word=""
    prev_word=[]
    for tok in nlp(sent):
        if tok.dep_.find('nsubj') != -1: #Found the subject of the sentence
            if len(combined_word)==0:
                se= tok.text
            else:
                se= combined_word+" "+tok.text
            combined_word= ""
        else:
            combined_word= combined_word+" "+tok.text
        
    se=se.strip()
    r=r.strip()
    te=te.strip()
    return([se,r,te])


In [ ]:
#Getting set of triples (soruce, relation and target)
triple=[]
i=1
for sent in data['sentences']:
    triple.append(getTriple(sent))
for t in triple:
    print(i,")",t)
    i+=1

In [ ]:
#for each sentence get token, pos, tag, dependency and head
from pathlib import Path
from tabulate import tabulate
import pandas as pd
file_name='Sentence_Details.txt'
output_path = Path("/content/drive/MyDrive/Project/" + file_name)
with open(output_path,"w") as external_file:
    i=1
    for sent in data['sentences']:
        table_data=[]
        d= nlp(sent)
        for tok in d:
            table_data.append([tok.text,tok.pos_,tok.tag_,tok.dep_,tok.head])
        #print('\033[1m'+sent+'\033[0m', file=external_file)
        print(i,")\t",sent, file=external_file)
        print(tabulate(table_data,headers=['Token','POS','Tag','Dependency','Head']),file=external_file)
        print('\n',file=external_file)
        i+=1
    external_file.close()
    #df= pd.DataFrame(table_data, columns=['Token','POS','Tag','Dependency','Head'])
    #print(df)

In [ ]:
t= "The PMAM informs about the entitlements from the EHCP to the PM-JAY Beneficiary"
t2= "The SHA/NHA studies the feedback from the PMAM"
d= nlp(t)
e=nlp(t2)
displacy.render(d, style='dep', jupyter= True)
for tok in d:
    #print(tok.text)
    print('{0}\t\t\t|{1}|{2}|{3}|{4}'.format(tok.text,tok.tag_,tok.dep_,tok.head,spacy.explain(tok.tag_)))
    #print(tok.text,":(tag)",tok.tag_,"->",spacy.explain(tok.dep_),":(dep)",tok.dep_,":(head)",tok.head)
#displacy.render(e, style='dep', jupyter= True)
#for tok in e:
#    print(tok.text,":(tag) ",tok.tag_,":(dep) ",tok.dep_,":(head) ",tok.head)

In [ ]:
#Pick up from here
#Make Entity and Relation Extractor from the dependencies extracted from the text
#These dependencies are saved as images in GDrive Have a look at them

#if the dependency is nsubj then extract it along with its components as Source entity
#if the dependency is ROOT the extract it along with its components as Relation
#if the dependency is dobj then extract it along with its components as First Target Entity
#if the dependency is pobj then extract it along with its components as Second Target Entity

In [ ]:
#saving the dependency in each sentence as a image in the drive
#from pathlib import Path
#i=1
for sent in data['sentences']:
    d= nlp(sent)
    svg= displacy.render(d,style='dep',jupyter=True)
    #file_name = str(i) + ".svg"
    #output_path = Path("/content/drive/MyDrive/Project/images/" + file_name)
    #output_path.open("w", encoding="utf-8").write(svg)
    #i= i+1

In [ ]:
'''#Tokenize each sentence, remove punctuations, lemmatize and lower case
#Get Refined Sentences
refined_sent_list=[]
punctuations= {".",",","\n"," "}
for each_sent in data['sentences']:
    d= nlp(each_sent)
    refined_sent=""
    for token in d:
        if (token.text not in punctuations):
            #refined_sent= refined_sent+((token.lemma_).lower())+" " #lemmatize and lower case
            #refined_sent= refined_sent+((token.text).lower())+" " #lowercase
            refined_sent= refined_sent+(token.text)+" " #no changes
    refined_sent= refined_sent.strip()
    refined_sent_list.append(refined_sent)
for i in refined_sent_list:
    print(i)'''

In [ ]:
'''from spacy import displacy
#Check for each sentence the POS tagging and the dependencies
i= refined_sent_list[0]
print(i)
d= nlp(i)
for tok in d:
    print(tok.text,"->",tok.pos_,"->",tok.tag_,":",spacy.explain(tok.tag_))'''

In [ ]:
'''displacy.render(d, style='dep', jupyter=True)
for tok in d:
    print(tok.text,"--->",tok.dep_)'''

In [ ]:
'''t="The Patient approaches the EHCP Registration Desk"
d2= nlp(t)
displacy.render(d2, style='dep', jupyter=True)
for tok in d2:
    print(tok.text,"--->",tok.dep_,":",spacy.explain(tok.dep_))
#print("Entities:")
#for ent in d2.ents:
#    print(ent.text,": ",ent.label_,": ",spacy.explain(ent.label_))'''

In [ ]:
'''for sent in refined_sent_list:
    d= nlp(sent)
    for tok in d:
        print(tok.text,"->",tok.pos_,"->",tok.tag_,":",spacy.explain(tok.tag_))
    for ent in d.ents:
        print(ent,": ",ent.label_,": ",spacy.explain(ent.label_))'''